In [ ]:
from confluent_kafka import Producer, KafkaError
import json
import ccloud_lib
import tweepy
import json
import socket

In [ ]:
consumer_key    = ""
consumer_secret = ""
access_token    = ""
access_token_secret   = ""
bearer_token = ""

In [ ]:
class MyListener(tweepy.Stream):    
    
    # Callback function from kafka producer
    def acked(self,err, msg):
        if err is not None:
            print("Error = {}".format(err))
        else:
            print(f"Produced record to topic {msg.topic()} partition {msg.partition()} offset {msg.offset()}")
    
    
    def startKafkaProducer(self):
        config_file = 'path to your producer config file'
        conf = ccloud_lib.read_ccloud_config(config_file)
        
        # Create Producer instance
        producer_conf = ccloud_lib.pop_schema_registry_params_from_config(conf)
        producer_conf['acks']="all"
        self.producer = Producer(producer_conf)
    
    
    # Each time MyListener receives a tweet, it decodes it first by using utf-8
    # Then, send it to kafka confluent cloud to faker topic
    def on_data(self, data):
        try:
            msg = data.decode('utf-8')
            self.producer.produce('faker',value=msg,on_delivery=self.acked)
            self.producer.poll(0)
            self.producer.flush()
            return True
        except BaseException as e:
            print("Error on_data : {}".format(e))
        return True
    
    # Callback function if any error of the Twitter Streaming API
    def on_error(self,status):
        print(status)
        return True

In [ ]:
mylistener = MyListener(
    consumer_key,
    consumer_secret,
    access_token,
    access_token_secret
)

In [ ]:
mylistener.startKafkaProducer()

In [ ]:
mylistener.filter(track=["bitcoin","usa","ethereum","china","russia","iphone"])